In [ ]:
import re
import os

import requests
from io import StringIO

import pandas as pd
import geopandas as gpd

from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

os.environ['PROJ_DATA'] = '/perc10/data/miniconda3/envs/snow_viz/share/proj'

In [ ]:
def convert_columns_lowercase(df):
    """
    Renames DataFrame columns to be lowercase to be compliant with
    PostgrSQL naming convetions.
    """
    df.columns = [
        re.sub(r'(?<!^)([A-Z])', r'_\1', col).lower()
        for col in df.columns
    ]
    return df

In [ ]:
config = dict(
    drivername="postgresql+psycopg",
    query={"service": "swe_db"},
)
url = URL.create(**config)
engine = create_engine(url)

## Station Locations

In [ ]:
STATION_REQUEST = "https://wcc.sc.egov.usda.gov/awdbRestApi/services/v1/stations?stationTriplets=%2A%3AWY%3ASNTL%2C%2A%3AUT%3ASNTL%2C%2A%3ACO%3ASNTL&returnForecastPointMetadata=false&returnReservoirMetadata=false&returnStationElements=false&activeOnly=true"

In [ ]:
SNTL = requests.get(STATION_REQUEST).text

In [ ]:
SNTL = pd.read_json(StringIO(SNTL))

In [ ]:
SNTL = gpd.GeoDataFrame(
    SNTL,
    geometry=gpd.points_from_xy(SNTL.longitude, SNTL.latitude),
    crs="EPSG:4326",
)

In [ ]:
SNTL.drop(columns=['pedonCode', 'stateCode', 'dcoCode', 'stationId', 'networkCode', 'latitude', 'longitude'], inplace=True)

SNTL = convert_columns_lowercase(SNTL)

SNTL["begin_date"] = pd.to_datetime(SNTL["begin_date"])
SNTL["end_date"] = pd.to_datetime(SNTL["end_date"])

SNTL["station_triplet"] = SNTL["station_triplet"].astype("string")
# Don't use PostgreSQL keywords: "name"
SNTL.rename(columns={"name": "station_name"}, inplace=True)
SNTL["station_name"] = SNTL["station_name"].astype("string")
SNTL["county_name"] = SNTL["county_name"].astype("string")
SNTL["shef_id"] = SNTL["shef_id"].astype("string")
SNTL["operator"] = SNTL["operator"].astype("string")

In [ ]:
SNTL.head()

In [ ]:
SNTL.to_postgis("snotel_sites", engine, if_exists="replace")